In [2]:
import streamlit as st
import openai
import getpass
import os
from docx import Document
import csv
import smtplib
import ssl
from email.message import EmailMessage
from email.mime.text import MIMEText
import time
import random
import tiktoken
import pandas as pd
import numpy as np
import re

In [3]:
# Store API key
openai.api_key = 'YOUR API KEY'

# Store Email key (MAY DROP THIS IN FUTURE)
email_password = 'YOUR EMAIL KEY'

In [ ]:
# Custom function for converting text strings to tokens
def num_tokens_from_string(string, encoding_name):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Main function for evaluating document text
def process_docx_files(submissions_path, docx_files_list, rubric_content):
    results = {}
    count = 0
    for docx_file in docx_files_list:
        count += 1
        file_path = os.path.join(submissions_path, docx_file)
        
        # Open the .docx file using the Document class
        doc = Document(file_path)
        
        # Initialize a string to store the text content
        text_content = ""
        
        # Iterate through paragraphs in the document and append to text_content
        for paragraph in doc.paragraphs:
            text_content += paragraph.text + "\n"
            
        tokens = num_tokens_from_string(text_content,"cl100k_base")
        
        if tokens <= 6000:
            messages = [
                {'role': 'system', 'content': rubric_content},
                {'role': 'user', 'content': text_content}
                ]

            # roles => system, user, assistant
            # system message is to tell the bot the role it is supposed to play
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-16k',
                messages=messages,
                temperature=0.1,
                max_tokens=800
            )
        
            # Append results dictionary
            results[docx_file] = response
            
        else:
            results[docx_file] = "TOO LARGE TO PROCESS!!!"
    
    return results

def parse_results_to_dataframe(data):
    """
    Parse the results from the OpenAI API into a pandas DataFrame.
    
    Parameters:
    - data (dict): Dictionary containing file names as keys and OpenAI API responses as values.
    
    Returns:
    - DataFrame: A pandas DataFrame containing parsed results.
    """
    
    # This list will hold individual records that will be used to create the DataFrame
    records = []

    for key, value in data.items():
        record = {'student': key[0:9]}
        if isinstance(value, str) and value == 'TOO LARGE TO PROCESS!!!':
            # Handle the case where the value is 'TOO LARGE TO PROCESS!!!'
            record.update({
                "Feedback": None
            })
        else:
            # Parse content from JSON-like structure
            content = value["choices"][0]["message"]["content"]
            # Split the string based on headers

            record.update({
                "Feedback": content
            })

        records.append(record)

    return pd.DataFrame(records)

def main():
    st.title("AI Evaluator")
    
    # Folder selector
    folder_path = st.text_input("Enter folder path:")
    
    if folder_path:
        # Check for submissions folder and required CSV files
        if verify_files(folder_path):
            # Process Files Button
            if st.button("Process Files"):
                # Define the folder paths based on user input
                submissions_path = os.path.join(folder_path, 'submissions')
                assignment_path = folder_path
                
                # ... (rest of your logic for processing files, ensuring to adjust paths)
                # Get a list of all .docx files in the folder
                docx_files = [filename for filename in os.listdir(submissions_path) if filename.endswith('.docx') and not filename.startswith('~$')]

                # Get Rubric
                rubric_path = os.path.join(assignment_path, "Rubric.docx")

                # Open the .docx file using the Document class
                rubric = Document(rubric_path)

                # Initialize a string to store the text content
                instructions = ""

                # Iterate through paragraphs in the document and append to text_content
                for paragraph in rubric.paragraphs:
                    instructions += paragraph.text + "\n"

                # Process text from files
                start = time.time()
                results = process_docx_files(submissions_path,docx_files,instructions)
                end = time.time()
                elapsed_time = end-start

                # Extract AI Assessment and feedback
                df_new = parse_results_to_dataframe(results)

                
                # Read in student information
                students = pd.read_csv(os.path.join(assignment_path,"students.csv"))

                # Merge results with student information
                final_df = students.merge(df_new, on='student',how='left')
                
                st.success("Files processed successfully!")
                
                # Save the dataframe to the session state
                st.session_state.final_df = final_df

            # Check if final_df exists in session state
            if 'final_df' in st.session_state:
                # Dropdown to select a student and display their feedback
                student_list = st.session_state.final_df['student'].tolist()
                selected_student = st.selectbox("Select a student:", student_list)
                student_feedback = st.session_state.final_df[st.session_state.final_df['student'] == selected_student]['Feedback'].iloc[0]
                st.text_area("Feedback:", value=student_feedback, height=400)


            # Email Sending Button
            if st.button("Send Emails"):
                # ... (your logic for sending emails)
                # SMTP setup for email sending
                smtp_server = 'smtp.gmail.com'
                smtp_port = 465
                smtp_user = 'ai_eval@email.com'
                smtp_password = email_password

                # Send the emails
                # Function to send email
                def send_email(to_email, content,subject):
                    msg = MIMEText(content)
                    msg['Subject'] = subject
                    msg['From'] = smtp_user
                    msg['To'] = to_email

                    context = ssl.create_default_context()

                    with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as smtp:
                        smtp.login(smtp_user, smtp_password)
                        smtp.sendmail(smtp_user, to_email, msg.as_string())

                start_time = time.time()

                final_index = None

                for index, row in st.session_state.final_df.iterrows():
                    email_address = row['email']
                    subject = "AI Response to Homework: " + row['Feedback'][0:20] + "...."
                    content = row['Feedback']
                    if email_address:
                        send_email(email_address, content, subject)
                        time.sleep(random.uniform(10,20))
                    final_index = index

                end_time = time.time()
                elapsed_time = end_time - start_time
                st.success("Emails sent successfully!")
        else:
            st.error("The specified folder does not contain the required files and/or folders.")

def verify_files(folder_path):
    # Check for the existence of 'submissions' folder, 'Rubric.csv', and 'students.csv' in the specified folder.
    return all(os.path.exists(os.path.join(folder_path, name)) for name in ['submissions', 'Rubric.docx', 'students.csv'])

if __name__ == "__main__":
    main()


## Contribution

This project includes significant contributions from Dr. Geoff Pofahl, whose expertise and insights have been invaluable in shaping the development and implementation of the AI evaluator system.